In [2]:
# Install PySpark
!pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=cf37683f83a93811b3220b94bd126f784feb76fa148e1c36add113a549ebfde4
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [3]:
# Import necessary libraries
from google.colab import files
import pandas as pd
import io
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, stddev_samp, count, mean
from scipy.stats import chi2_contingency

In [4]:
# Set Up PySpark in Google Colab

In [5]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install pyspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Comorbidity Analysis").getOrCreate()


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,910 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [2,469 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/mai

In [6]:
# Upload and Load Data

In [9]:
# Upload file
uploaded = files.upload()

# Print the keys to check the exact filename
print(uploaded.keys())

# Use the correct key to read the CSV file into a pandas DataFrame
file_key = list(uploaded.keys())[0]
merged_df = pd.read_csv(io.BytesIO(uploaded[file_key]))

# Initialize Spark session
spark = SparkSession.builder.appName("comorbidity_results.csv").getOrCreate()

# Convert pandas DataFrame to Spark DataFrame
spark_df = spark.createDataFrame(merged_df)

Saving Cleaned_dataset.csv to Cleaned_dataset (2).csv
dict_keys(['Cleaned_dataset (2).csv'])


In [10]:
# List of comorbidities
comorbidities = ['HadAngina', 'HadStroke', 'HadCOPD', 'HadKidneyDisease', 'HadArthritis', 'HadDiabetes']

# Prepare the data for plotting
results = []

for comorbidity in comorbidities:
    if comorbidity in spark_df.columns:
        # Calculate proportions and confidence intervals
        proportion_df = spark_df.groupBy(comorbidity).agg(
            mean('HadHeartAttack').alias('mean'),
            count('HadHeartAttack').alias('count'),
            stddev_samp('HadHeartAttack').alias('std')
        )

        proportion_df = proportion_df.withColumn(
            'ci', 1.96 * (col('std') / (col('count') ** 0.5))
        ).toPandas()

        # Perform Chi-square test
        contingency_table = spark_df.crosstab(comorbidity, 'HadHeartAttack').toPandas()
        chi2, p_value, _, _ = chi2_contingency(contingency_table.values[:, 1:])

        for i, row in proportion_df.iterrows():
            results.append({
                'comorbidity': comorbidity,
                'level': row[comorbidity],
                'mean': row['mean'],
                'ci': row['ci'],
                'chi2': chi2,
                'p_value': p_value
            })

# Convert results to DataFrame and save to CSV
results_df = pd.DataFrame(results)
results_df.to_csv("comorbidity_results.csv", index=False)

# Download the CSV file to your local machine
files.download("comorbidity_results.csv")

# Stop Spark session
spark.stop()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>